# Advent of Code 2021 - Day 3

In [1]:
data class DiagnosticReport(val numbers: String, val chunk: Int)

In [2]:
import java.io.File

val regex = Regex("""[^1,0]""")

val report: DiagnosticReport = File("Day3.input.txt")
    .bufferedReader()
    .readText()
    .let { DiagnosticReport(numbers = regex.replace(it, ""), chunk = it.indexOf('\n') - 1) }

## Part 1

Calculate the power consumption from the `DiagnosticReport` by multiplying the calculated gamma by the calculated epsilon. Calculate the gamma by taking the bit that appears the most in its position split by `chunk` and converting it to decimal. Epsilon is calculated similarly, except it is the bit that appears the least.

In [3]:
val DiagnosticReport.chunkedByPosition: List<String>
    get() = (0 until chunk).map { position ->
        numbers.foldIndexed("") { index, acc, char ->
            if (index % chunk == position) acc + char else acc
        }
    }

fun List<String>.foldByCharGroup(fn: (String, Map<Char, List<Char>>) -> String): String =
    map { string -> string.groupBy { it } }
    .fold("") { string, group -> fn(string, group) }

val DiagnosticReport.gamma: Int
    get() = chunkedByPosition.foldByCharGroup { gamma, group -> 
        gamma + group.maxByOrNull { it.value.size }!!.key
    }
    .toInt(2)

val DiagnosticReport.epsilon: Int
    get() = chunkedByPosition.foldByCharGroup { epsilon, group -> 
        epsilon + group.minByOrNull { it.value.size }!!.key
    }
    .toInt(2)

val DiagnosticReport.powerConsumption: Int 
    get() = gamma * epsilon

report.powerConsumption

1997414

### Notes

I think there's a ton of ways to solve this problem. For instance notice that `epsilon` is just the bitwise inverse of `gamma`. However, Kotlin's built in bitwise inverse (`Int::inv`) returns the two's compliment. I'm pretty sure there's also a really clever way to solve this problem, but 🤷. Also my `report` input was definitely influenced by the problem statement, it would probably be more flexible as a list of strings (or integers).

## Part 2

Calculate the life support rating from the `DiagnosticReport` by multiplying the calculated oxygen generator rating and the CO2 scrubber rating. The oxygen generator rating is calculated by checking every position for the bit that appears the most and then only keeping entries that contain that number in that position. This is repeated until there is only one number left. CO2 scrubber rating is calculated similarly, except you keep the least appearing bit.

In [4]:
val DiagnosticReport.numbersList: List<String>
    get() = report.numbers.chunked(report.chunk)

val DiagnosticReport.matrix: List<List<Int>>
    get() = numbersList.map { string -> string.map { it.digitToInt(2) }}

fun List<List<Int>>.countColumn(column: Int): Map<Int, Int> = fold(mapOf()) { acc, row ->
    val bit = row[column]
    acc + (bit to (acc[bit]?.plus(1) ?: 1))
}

fun List<List<Int>>.calculateRating(keepCondition: (Map<Int, Int>) -> Int): Int =
    (0 until this[0].size).fold(this) { acc, index ->
        if (acc.size == 1) acc
        else acc.countColumn(index)
            .let { keepCondition(it) }
            .let { keep -> acc.filter { it[index] == keep } }
    }[0]
    .joinToString("")
    .toInt(2)

fun Int?.gte(other: Int?): Boolean {
    return if (this == null && other == null) true
    else if (this != null && other == null) true
    else if (this == null && other != null) false
    else {
        this!! >= other!!
    }
}

val DiagnosticReport.oxygenGeneratorRating: Int
    get() = matrix.calculateRating { if (it[1].gte(it[0])) 1 else 0 }

val DiagnosticReport.co2ScrubberRating: Int
    get() = matrix.calculateRating { if (it[1].gte(it[0])) 0 else 1 }

val DiagnosticReport.lifeSupportRating: Int
    get() = oxygenGeneratorRating * co2ScrubberRating

report.lifeSupportRating

1032597

### Notes

This problem was really really really long to parse, and at some point I just got tired. I think in retrospect the `matrix` I created for part 2 could have simplified the solution in part 1 a lot. Also it turns out that there's not really a good way to compare nullable integers, but I guess it makes sense when you consider the ways you can handle the null scenarios.

Overall, not really happy with this day.